# Importing Required Libraries 

First, let's install ultralytics, which contains YOLOv8!

In [1]:
# Clears the output
!rm -rf /kaggle/working/*

In [2]:
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.5/825.5 kB 25.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


Now we can install the necessary components we will be using.

In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
# from matplotlib import animation, rc
# rc('animation', html='jshtml')
from PIL import Image

import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.2.66 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 5771.7/8062.4 GB disk)


In [4]:
path = os.path.relpath("/kaggle/input/maze-classification-localization/data")
files = os.listdir(path)
files

['valid', 'test', 'train']

# Create yaml file
YAML is a data serialization language that is designed to be 'human readable'

In [5]:
import yaml

data_yaml = dict(
    train='train',
    val='valid',
    test='test',
    nc=1,
    names=['distortion']
)

with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat data.yaml

{names: [distortion], nc: 1, test: test, train: train, val: valid}


In [6]:
shutil.copytree("/kaggle/input/maze-classification-localization/data", "/kaggle/working/datasets", dirs_exist_ok=True)
os.chdir("/kaggle/working")

# Train
Now we can begin looking into training the model.  First let's load the model.

In [7]:
model = YOLO("yolov8x.pt")

100%|██████████| 131M/131M [00:01<00:00, 115MB/s]


In [8]:
# model.names = {0: 'distortion'}
# model.nc = 1

In [9]:
# !yolo task=detect mode=train model=yolov8x.pt data=data.yaml epochs=4 imgsz=480
results = model.train(data='data.yaml', epochs=4, imgsz=480, name='bulge_yolo_model')

Ultralytics YOLOv8.2.66 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=data.yaml, epochs=4, time=None, patience=100, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=bulge_yolo_model, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, l

100%|██████████| 755k/755k [00:00<00:00, 19.2MB/s]
2024-07-26 01:40:44,422	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-07-26 01:40:45,228	INFO util.py:124 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics

100%|██████████| 6.25M/6.25M [00:00<00:00, 116MB/s]


AMP: checks passed ✅


...

        4/4      8.55G     0.3101     0.2846     0.9546          5        480: 100%|██████████| 300/300 [03:25<00:00,  1.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:09<00:00,  1.92it/s]

                   all        600        300          1          1      0.995      0.992



4 epochs completed in 0.247 hours.
Optimizer stripped from runs/detect/bulge_yolo_model/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/bulge_yolo_model/weights/best.pt, 136.7MB

Validating runs/detect/bulge_yolo_model/weights/best.pt...
Ultralytics YOLOv8.2.66 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 19/19 [00:10<00:00,  1.77it/s]


                   all        600        300          1          1      0.995      0.992
Speed: 0.1ms preprocess, 14.0ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs/detect/bulge_yolo_model


# Saving the Model

Now we are going to save the model! We can use the built in `.save` function within ultralytics.  We will save it with the .pt extension which is a PyTorch extension.  

In [10]:
model.predict('/kaggle/input/maze-classification-localization/data/train/images/1004_distorted.jpg')


image 1/1 /kaggle/input/maze-classification-localization/data/train/images/1004_distorted.jpg: 480x480 1 distortion, 55.8ms
Speed: 1.8ms preprocess, 55.8ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 480)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'distortion'}
 obb: None
 orig_img: array([[[236, 236, 236],
         [255, 255, 255],
         [251, 251, 251],
         ...,
         [246, 246, 246],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [234, 234, 234],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[235, 235, 235],
         [246, 246, 246],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [245, 245, 245],
         [242, 242, 242]],
 
        ...,
 
        [[245, 245, 245],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [254, 254, 254],
         [250, 250, 250]],
 
        [[252, 252, 252],
         [253, 253, 253],
         [252, 252, 252],
         ...,
         [251,

In [11]:
# model.save('/kaggle/working/bulge_yolo_model.pt')